In [1]:
import requests
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
sia =SentimentIntensityAnalyzer()
def get_sentiment_of(sentence_in):
    try:
        scrs=sia.polarity_scores(sentence_in)
        the_important_one=scrs['compound']
    except:
        the_important_one=0
    return the_important_one

In [3]:
import pandas as pd
import csv

df = pd.read_csv("output_meta_yelpResData_NRYRcleaned.txt", delimiter = " ", names = [ "Date" , "review ID", "reviewer ID", "product ID", "Label", "Unknown1", "Unknown2", "reviewUsefulCount", "Star Rating"])
df2 = pd.read_csv("output_review_yelpResData_NRYRcleaned.txt", delimiter = "\n",names = ["review"])

In [4]:
result = pd.concat([df, df2], axis = 1)

In [5]:
f = open("output_review_yelpResData_NRYRcleaned.txt", "r")
sent = list()
for rows in f:
    #print(rows)
    sent.append(get_sentiment_of(rows))
print(len(sent))

61541


In [6]:
import numpy as np

result['Sentiment'] = sent
df_final = result.drop(['review ID','product ID', 'reviewer ID', 'Date', 'review' ], axis = 1)
authentic_reviews_df = df_final[df_final['Label'] == 'N']
fake_reviews_df = df_final[df_final['Label'] == 'Y']
authentic_reviews_df = authentic_reviews_df.sample(n = 8141)

In [7]:
df_final = pd.concat([authentic_reviews_df , fake_reviews_df])
df_final['Label'] = [0 if x == 'N' else 1 for x in df_final['Label']]

In [8]:
flagged = df_final['Label']
df_final = df_final.drop(['Label'], axis=1)
df_final = pd.concat([df_final,flagged], axis=1)
X = df_final.iloc[:, :-1]
y = flagged
X.head()

,Unknown1,Unknown2,reviewUsefulCount,Star Rating,Sentiment
14612,0,0,0,4,0.9718
4914,0,0,2,4,0.9891
24963,0,0,0,5,0.9682
48376,1,2,2,5,0.9952
21955,0,1,1,4,0.9424


In [9]:
X = X.values

In [10]:
y = yreshape(-1,1)

In [11]:
#Normalizing the data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [12]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()

C:\Users\Euclid\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.1)

In [14]:
#Dependencies
import keras
from keras.models import Sequential
from keras.layers import Dense

# Neural network
model = Sequential()
model.add(Dense(16, input_dim=5, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(2, activation='softmax'))

Using TensorFlow backend.


In [1]:
model.compile(loss='sparse_categorical_crossenropy', optimizer='adam', metrics=['accuracy'])

NameError: name 'model' is not defined

In [16]:
history = model.fit(X_train, y_train, epochs=100, batch_size=64)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/100
14653/14653 [==============================] - 1s 39us/step - loss: 0.5944 - acc: 0.6860
Epoch 2/100
14653/14653 [==============================] - 0s 15us/step - loss: 0.4929 - acc: 0.7484
Epoch 3/100
14653/14653 [==============================] - 0s 17us/step - loss: 0.4691 - acc: 0.7505
Epoch 4/100
14653/14653 [==============================] - 0s 16us/step - loss: 0.4628 - acc: 0.7512
Epoch 5/100
14653/14653 [==============================] - 0s 18us/step - loss: 0.4602 - acc: 0.7510: 0s - loss: 0.4661 - acc: 0.
Epoch 6/100
14653/14653 [==============================] - 0s 15us/step - loss: 0.4585 - acc: 0.7545
Epoch 7/100
14653/14653 [==============================] - 0s 15us/step - loss: 0.4572 - acc: 0.7540
Epoch 8/100
14653/14653 [==============================] - 0s 17us/step - loss: 0.4568 - acc: 0.7548
Epoch 9/100
14653/14653 [==============================] - 0s 16us/s

Epoch 75/100
14653/14653 [==============================] - 0s 16us/step - loss: 0.4541 - acc: 0.7558
Epoch 76/100
14653/14653 [==============================] - 0s 16us/step - loss: 0.4540 - acc: 0.7566
Epoch 77/100
14653/14653 [==============================] - 0s 15us/step - loss: 0.4539 - acc: 0.7558
Epoch 78/100
14653/14653 [==============================] - 0s 15us/step - loss: 0.4536 - acc: 0.7566
Epoch 79/100
14653/14653 [==============================] - 0s 16us/step - loss: 0.4540 - acc: 0.7552
Epoch 80/100
14653/14653 [==============================] - 0s 16us/step - loss: 0.4534 - acc: 0.7572
Epoch 81/100
14653/14653 [==============================] - 0s 16us/step - loss: 0.4540 - acc: 0.7555
Epoch 82/100
14653/14653 [==============================] - 0s 16us/step - loss: 0.4540 - acc: 0.7567
Epoch 83/100
14653/14653 [==============================] - 0s 16us/step - loss: 0.4539 - acc: 0.7577
Epoch 84/100
14653/14653 [==============================] - 0s 16us/step - loss: 0

In [17]:
y_pred = model.predict(X_test)
#Converting predictions to label
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
#Converting one hot encoded test label to label
test = list()
for i in range(len(y_test)):
    test.append(np.argmax(y_test[i]))

In [18]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

a = accuracy_score(test, pred)
print('Accuracy is:', a*100)
print(classification_report(test, pred))



Accuracy is: 74.52424800491099
              precision    recall  f1-score   support

           0       0.93      0.54      0.68       834
           1       0.67      0.96      0.79       795

   micro avg       0.75      0.75      0.75      1629
   macro avg       0.80      0.75      0.74      1629
weighted avg       0.80      0.75      0.73      1629



In [19]:
from matplotlib import pyplot as plt

plt.plot(history.history['loss']) 
print(history.history.keys())
plt.title('Model loss') 
plt.ylabel('Loss') 
plt.xlabel('Epoch') 
plt.legend(['Train', 'Test'], loc='upper left') 
plt.show()

dict_keys(['loss', 'acc'])


<Figure size 640x480 with 1 Axes>